<!-- Banner Image -->
<img src="https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdevnotebooks.png" width="100%">

<!-- Links -->
<center>
  <a href="https://console.brev.dev" style="color: #06b6d4;">Console</a> •
  <a href="https://brev.dev" style="color: #06b6d4;">Docs</a> •
  <a href="/" style="color: #06b6d4;">Templates</a> •
  <a href="https://discord.gg/NVDyv7TUgJ" style="color: #06b6d4;">Discord</a>
</center>

# Fine-tune and deploy the multimodal LLaVA model with DeepSpeed🤙

Hi everyone!

In this notebook we'll fine-tune the LLaVA model. LLaVA is multimodal which means it can ingest and understand images along with text! LLaVA comes from a research paper titled [Visual Instruction Tuning](https://arxiv.org/abs/2304.08485) and introduces the Large Language and Vision Assistant methodology. In order to process images, LLaVA relies on the pre-trained CLIP visual encoder ViT-L/14 which maps images and text into the same latent space. 

Help us make this tutorial better! Please provide feedback on the [Discord channel](https://discord.gg/RN2a436M73) or on [X](https://x.com/brevdev).

## Table of contents 

1. Data Preprocessing
2. LLaVA Installation
3. DeepSpeed configuration
4. Weights and Biases
5. Finetuning flow
6. Deployment via gradio interface

## Data Preprocessing 

LLaVA requires data to be in a very specific format. Below we use a [helper function](https://wandb.ai/byyoung3/ml-news/reports/How-to-Fine-Tune-LLaVA-on-a-Custom-Dataset--Vmlldzo2NjUwNTc1) to format the OKV-QA dataset. This dataset teaches the model to respond to an image in short phrases without any preamble or extra verbiage. 

## Back to it!

In [ ]:
#Import data here
!pip install gdown
!gdown --id 1KaK3iv23ULq0rFfF_OvN9XCgAjic6VZz #HAM10000_balanced

In [ ]:
#Unzip data here
from zipfile import ZipFile
file_name = "HAM10000_balanced.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
# Install preprocessing libraries
!pip install datasets
!pip install --upgrade --force-reinstall Pillow
#this was added
!pip install torch==2.2.0

## Install LLaVA

To install the functions needed to use the model, we have to clone the original LLaVA repository and and install it in editable mode. This lets us access all functions and helper methods 

In [ ]:
# The pip install -e . lets us install the repository in editable mode
!git clone https://github.com/haotian-liu/LLaVA.git
!cd LLaVA && pip install --upgrade pip && pip install -e .

## DeepSpeed

Microsoft DeepSpeed is a deep learning optimization library designed to enhance the training speed and scalability of large-scale artificial intelligence (AI) models. Developed by Microsoft, this open-source tool specifically addresses the challenges associated with training very large models, allowing for reduced computational times and resource usage. By optimizing memory management and introducing novel parallelism techniques, DeepSpeed enables developers and researchers to train models with billions of parameters efficiently, even on limited hardware setups.DeepSpeed API is a lightweight wrapper on PyTorch. DeepSpeed manages all of the boilerplate training techniques, such as distributed training, mixed precision, gradient accumulation, and checkpoints and allows you to just focus on model development. To learn more about DeepSpeed and how it performs the magic, check out this [article](https://www.deepspeed.ai/2021/03/07/zero3-offload.html) on DeepSpeed and ZeRO.

Using deepspeed is extremely simple - you simply pip install it! The LLaVA respository contains the setup scripts and configuration files needed to finetune in different ways. 

In [ ]:
!cd LLaVA && pip install -e ".[train]"
#version specified here
!pip install flash-attn==2.7.3 --no-build-isolation

In [ ]:
!pip install deepspeed

## Weights and Biases

Weights and Biases is an industry standard MLOps tool to used to monitor and evaluate training jobs. At Brev, we use Weights and Biases to track all of our finetuning jobs! Its extremely easy to setup and plugs into the DeepSpeed training loop. You simply create an account and use the cells below to log in!

In [ ]:
!pip install wandb

In [ ]:
import wandb

wandb.login()

In [ ]:
#this was added for next cell to work
!pip install peft==0.10.0

## Finetuning job

Below we start the DeepSpeed training run for 5 epochs. It will automatically recognize multiple GPUs and parallelize across them. Most of the input flags are standard but you can adjust your training run with the `num_train_epochs` and `per_device_train_batch_size` flags!

In [ ]:
!deepspeed LLaVA/llava/train/train_mem.py \
    --lora_enable True --lora_r 128 --lora_alpha 256 --lora_dropout 0.05 --mm_projector_lr 2e-5 \
    --deepspeed LLaVA/scripts/zero3.json \
    --model_name_or_path liuhaotian/llava-v1.5-13b \
    --version v1 \
    --data_path ./dataset/train/dataset.json \
    --image_folder ./dataset/images \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir ./checkpoints/llava-v1.5-13b-task-lora \
    --num_train_epochs 5 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0.01 \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb

In [ ]:
# merge the LoRA weights with the full model
!python LLaVA/scripts/merge_lora_weights.py --model-path checkpoints/llava-v1.5-13b-task-lora --model-base liuhaotian/llava-v1.5-13b --save-model-path llava-ftmodel

In [ ]:
# bump transformers down for gradio/deployment inference if needed
!pip install transformers==4.37.2

## Deployment

LLaVA gives us 2 ways to deploy the model - via CLI or Gradio UI. We suggest using the Gradio UI for interactivity as you can compare two models and see the finetuning effect compared to the original model.

In [ ]:
# Uncomment the lines below to run the CLI. You need to pass in a JPG image URL to use the multimodal capabilities

# !python -m llava.serve.cli \
#     --model-path llava-ftmodel \
#     --image-file "https://llava-vl.github.io/static/images/view.jpg"

In [ ]:
# Download the model runner
!wget -L https://raw.githubusercontent.com/brevdev/notebooks/main/assets/llava-deploy.sh 

In [ ]:
#this was added for UI to work
%cd LLaVA 
!pip install gradio -U

In [ ]:
#back to home directory
%cd /home/ubuntu/verb-workspace

In [ ]:
# Run inference! Use the public link provided in the output to test
!chmod +x llava-deploy.sh && ./llava-deploy.sh